In [1]:
import os
import codecs
import re
from time import sleep

In [2]:
# get google cloud
!pip install --upgrade google-cloud-translate
from google.cloud import translate

Requirement already up-to-date: google-cloud-translate in c:\users\user\anaconda3\lib\site-packages (1.3.3)


In [3]:
# get nltk to calculate bleu score
!pip install --upgrade nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

Requirement already up-to-date: nltk in c:\users\user\anaconda3\lib\site-packages (3.4)


# Zulu

In [40]:
# load data
en_file = codecs.open('../clean/en_zu/enzu_parallel.test.en', 'r')
zu_file = codecs.open('../clean/en_zu/enzu_parallel.test.zu', 'r')

en_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in en_file.readlines()]
zu_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in zu_file.readlines()]

# translate
if not os.path.exists('./translated.zu'):
    trans_file = codecs.open('./translated.zu','w')
    
    # translate 20 sentences at a time
    translate_client = translate.Client()
    target = 'zu'
    for i in range(0, len(en_sents) // 20):
        translations = translate_client.translate(
            en_sents[i*20:(i+1)*20],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')

        print("Translated %d sentences" %((i+1)*20))
        sleep(1.5)
        
    trans_file.close()
    
# calculate bleu score
trans_file = codecs.open('./translated.zu', 'r')
trans_sents = [line.rstrip('\n').split(' ') for line in trans_file.readlines()]
num_trans = len(trans_sents)

zu_refs = [[re.sub(r'\s+([?.!,"])', r'\1', line).split(' ')] for line in zu_sents]

bleu_1 = corpus_bleu(zu_refs[:num_trans], trans_sents, weights=(1, 0, 0, 0))*100
bleu_4 = corpus_bleu(zu_refs[:num_trans], trans_sents)*100

print('BLEU1: %.3f' %bleu_1)
print('BLEU4: %.3f' %bleu_4)

trans_file.close()
en_file.close()
zu_file.close()

BLEU1: 27.090
BLEU4: 7.546


In [15]:
num_trans

2880

In [37]:
n = 65

print(en_sents[n])
print(zu_refs[n])
print(trans_sents[n])
print(sentence_bleu(zu_refs[n], trans_sents[n])*100)


CS .0171 Annual contract for the supply , delivery , installation , testing and commissioning of fibre network monitoring system for 12/24/36 month period Closing date: 18 May 2007
[['CS.0171', 'Inkontileka', 'yonyaka', 'yokuthengisela,', 'ukudiliva,', 'ukufaka,', 'ukuhlola', 'nokubeka', 'esimweni', 'sokusebenza', 'i-fibre', 'network', 'monitoring', 'system', 'izinyanga', 'ezingu', '12/24/36', 'Kuvalwa:', '18', 'Meyi', '2007']]
['CS', '.0171', 'Inkontileka', 'yonyaka', 'yokuhlinzeka,', 'ukulethwa,', 'ukufakwa,', 'ukuhlolwa', 'nokuthunyelwa', 'kohlelo', 'lokuqapha', 'kwenethiwekhi', 'ye-fibre', 'ngo', '12/24/36', 'inyanga', 'inyanga', 'yokuvalwa:', '18', 'Meyi', '2007']
2.6616657200018398e-76


In [107]:
zu_sents[0]

'Imibhalo yamathenda itholakala eThekwini Electricity , 1 Jelf Taylor Crescent , Durban ngo 08:00 kuya ku 15:00 , ngeMisombuluko kuya'

# Afrikaans

In [161]:
# load data
en_file = codecs.open('../clean/en_af/enaf_parallel.test.en', 'r', 'utf-8')
af_file = codecs.open('../clean/en_af/enaf_parallel.test.af', 'r', 'utf-8')

en_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in en_file.readlines()]
af_sents = [re.sub(' +', ' ', line.rstrip(' \n')) for line in af_file.readlines()]

# translate
if not os.path.exists('./translated.af'):
    trans_file = codecs.open('./translated.af','w', 'utf-8')
    
    # translate 20 sentences at a time
    translate_client = translate.Client()
    target = 'af'
    for i in range(0, len(en_sents) // 20):
        translations = translate_client.translate(
            en_sents[i*20:(i+1)*20],
            target_language=target,
            source_language='en'
        )

        for translation in translations:
            trans_file.write(translation['translatedText'])
            trans_file.write('\n')

        print("Translated %d sentences" %((i+1)*20))
        sleep(2)
        
    trans_file.close()
    
# processing for consistency between strings
trans_file = codecs.open('./translated.af', 'r')
trans_sents = [line.rstrip('\n') for line in trans_file.readlines()]
trans_sents = [re.sub(r'(&gt;)', r' >', line) for line in trans_sents]
trans_sents = [re.sub(r'Ã«', r'ë', line) for line in trans_sents]
trans_sents = [re.sub(r'&#39;', r"'", line) for line in trans_sents]
#trans_sents = [re.sub(r'\)]
trans_sents = [re.sub(r'(\w)([?.!,"/()])', r'\1 \2', line) for line in trans_sents]
trans_sents = [re.sub(r'([?.!,"/()])(\w)', r'\1 \2', line) for line in trans_sents]
trans_refs = [line.split(' ') for line in trans_sents]

num_trans = len(trans_sents)

af_sents = [re.sub(r'(\w)([?.!,"/()])', r'\1 \2', line) for line in af_sents]
af_sents = [re.sub(r'([?.!,"/()])(\w)', r'\1 \2', line) for line in af_sents]
af_refs = [[line.rstrip('\r').split(' ')] for line in af_sents]

# calculate bleu score
bleu_1 = corpus_bleu(af_refs[:num_trans], trans_refs, weights=(1, 0, 0, 0))*100
bleu_4 = corpus_bleu(af_refs[:num_trans], trans_refs)*100

print('BLEU1: %.3f' %bleu_1)
print('BLEU4: %.3f' %bleu_4)

trans_file.close()
en_file.close()
af_file.close()

BLEU1: 68.814
BLEU4: 41.181


In [157]:
n = 113

print(en_sents[n])
print(af_refs[n])
print(trans_refs[n])
print(sentence_bleu(af_refs[n], trans_sents[n])*100)


The Western Cape ARV programme aims to establish 36 accredited ARV treatment sites by March 2005 and to provide daily ARV treatment to 6,000 patients (estimated to be half of those in need of treatment) by March 2005.
[['Die', 'Wes-Kaapse', 'TRV-program', 'wil', 'teen', 'Maart', '2005', '36', 'geakkrediteerde', 'TRV-behandelingspunte', 'op', 'die', 'been', 'hê', 'en', 'daagliks', 'TRV-behandeling', 'aan', '6', ',', '000', 'pasiënte', 'verskaf', '.', '(Daar', 'word', 'beraam', 'dat', 'dit', 'die', 'helfte', 'van', 'die', 'mense', 'is', 'wat', 'behandeling', 'nodig', 'het', '.)']]
['Die', 'Wes-Kaapse', 'ARV-program', 'beoog', 'om', 'teen', 'Maart', '2005', '36', 'geakkrediteerde', 'ARV-behandelingsterreine', 'te', 'vestig', 'en', 'teen', 'Maart', '2005', 'elke', 'maand', '6', '000', 'pasiënte', '(na', 'raming', 'die', 'helfte', 'van', 'diegene', 'wat', 'behandeling', 'benodig),', 'te', 'voorsien', '.']
6.197644091669366e-230
